In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [2]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

In [3]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cum_2018_2024.csv"
df = pd.read_csv(folder_path, index_col=0)
df = df.reset_index(drop=True)

In [10]:
df['Teams']

0        BOS
1        GSW
2        OKC
3        PHI
4        ATL
        ... 
33747    SAC
33748    SAS
33749    TOR
33750    UTA
33751    WAS
Name: Teams, Length: 33752, dtype: object

In [11]:
date_prediction = '2024-04-05'


df = df[df['Date'] == '2024-04-04']
df['date_next'] = add_col(df, 'Date')
df['date_next'].fillna(date_prediction, inplace=True)
dates = df['date_next'].unique()

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_22303/138724945.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_22303/1686756084.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_next'] = add_col(df, 'Date')
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_22303/1686756084.py:6: F

In [12]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'Los Angeles Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [13]:
# Define time zones
dates_utc = []
est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
utc_timezone = pytz.timezone('UTC')

threshold_value = pd.to_datetime('2020-06-27T03:55:00Z')

full = []
for date in dates:
    date = date[:10]
    # Create a datetime object for the given time in EST
    est_time = datetime.strptime(f'{date} 4:00 PM', '%Y-%m-%d %I:%M %p')
    est_time = est_timezone.localize(est_time)

    # Convert EST time to UTC
    utc_time = est_time.astimezone(utc_timezone)
    if utc_time >= threshold_value:
        final_time = utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        dates_utc.append(final_time)

In [14]:
KEY = 'dc34ee156925bf51d4a4c33c87d440fc'
SPORT = 'basketball_nba'
REGION = 'us'
MARKET = 'h2h'
FORMAT = 'decimal'
full = []
# adjust number of dates 
DATES = dates_utc
for DATE in DATES:
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{SPORT}/odds',
        params={
            'api_key': KEY,
            'regions': REGION,
            'markets': MARKET,
            'oddsFormat': FORMAT,
            'date': DATE,
        }
    ).json()
    for i in range(len(response['data'])):
        # [Timestamp, good team, evil team, fanduel odds good team wins, draftkings odds, caesars odds]
        #best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
        bookmakers = response['data'][i]['bookmakers']
        for index, data in enumerate(bookmakers):
            best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
            if response['data'][i]['bookmakers'][index]['title'] in ["FanDuel", "BetMGM", "DraftKings", ""]:
                for outcome in range(2):
                    odds = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['price']
                    current_odds = best_odds[outcome][4]
                    best_odds[outcome][0] = response['timestamp']
                    best_odds[outcome][1] = response['data'][i]['id']
                    best_odds[outcome][2] = response['data'][i]['bookmakers'][index]['title']
                    best_odds[outcome][3] = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][4] = odds
                full.extend(best_odds)

In [15]:
timestamp = []
Id = []
sportsbook = []
team = []
odds = []
for i in range(len(full)):
    timestamp.append(full[i][0])
    Id.append(full[i][1])
    sportsbook.append(full[i][2])
    team.append(full[i][3])
    odds.append(full[i][4])
df = pd.DataFrame({"Timestamp": timestamp,"Id": Id, "Sportsbook": sportsbook, "Team": team, "Odds": odds})

In [16]:
df

,Timestamp,Id,Sportsbook,Team,Odds
0,2024-04-05T14:00:39Z,763d66b5c0bcee2bc43f90edc1155138,DraftKings,Charlotte Hornets,6.00
1,2024-04-05T14:00:39Z,763d66b5c0bcee2bc43f90edc1155138,DraftKings,Orlando Magic,1.14
2,2024-04-05T14:00:39Z,763d66b5c0bcee2bc43f90edc1155138,FanDuel,Charlotte Hornets,6.30
3,2024-04-05T14:00:39Z,763d66b5c0bcee2bc43f90edc1155138,FanDuel,Orlando Magic,1.13
4,2024-04-05T14:00:39Z,763d66b5c0bcee2bc43f90edc1155138,BetMGM,Charlotte Hornets,6.00
...,...,...,...,...,...
65,2024-04-05T14:00:39Z,6a200cbf8fb5f63c63c4d96b9a3a0ec6,DraftKings,Utah Jazz,6.75
66,2024-04-05T14:00:39Z,6a200cbf8fb5f63c63c4d96b9a3a0ec6,FanDuel,Los Angeles Clippers,1.11
67,2024-04-05T14:00:39Z,6a200cbf8fb5f63c63c4d96b9a3a0ec6,FanDuel,Utah Jazz,7.10
68,2024-04-05T14:00:39Z,6a200cbf8fb5f63c63c4d96b9a3a0ec6,BetMGM,Los Angeles Clippers,1.12


In [17]:
grouped = df.groupby('Id')
group_sizes = grouped.size()

# Filter groups with less than 6 rows
ids_with_less_than_six_rows = group_sizes[group_sizes < 6].index.tolist()

print("Ids with less than 6 rows:")

ids_with_less_than_six_rows

Ids with less than 6 rows:


['549cb43863d83987dac2acd5bed16a33']

In [18]:
miss = []
for Id in ids_with_less_than_six_rows:
    group = df[df['Id'] == Id]
    sportsbooks = ["FanDuel", "BetMGM", "DraftKings"]
    
    # check which sportsbook is missing
    present_sportsbooks = group['Sportsbook'].unique()
    missing_sportsbooks = [sb for sb in sportsbooks if sb not in present_sportsbooks][0]

    teams = group['Team'].unique()
    for team in teams:
        missing = [None, None, None, None, float('-inf')]
        # calculate average odds 
        missing[0] = group['Timestamp'].iloc[0]
        missing[1] = Id
        missing[2] = missing_sportsbooks
        missing[3] = team
        missing[4] = round(group[group['Team'] == team]['Odds'].mean(), 2)
        miss.append(missing)
missing_df = pd.DataFrame(miss, columns=['Timestamp', 'Id', 'Sportsbook', 'Team', 'Odds'])

In [19]:
concat = pd.concat([df, missing_df], ignore_index=True)
concat = concat.sort_values(by=['Timestamp', 'Id'])
concat = concat.reset_index(drop=True)
full = concat.merge(concat,
               left_on=['Timestamp','Id', 'Sportsbook'],
               right_on = ['Timestamp','Id', 'Sportsbook'])
full = full.drop(full[full['Team_x'] == full['Team_y']].index)
full = full.reset_index(drop=True)

In [20]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-05T14:00:39Z,1439dd82b2ac2434f097380dfa7c39aa,DraftKings,Minnesota Timberwolves,2.42,Phoenix Suns,1.59
1,2024-04-05T14:00:39Z,1439dd82b2ac2434f097380dfa7c39aa,DraftKings,Phoenix Suns,1.59,Minnesota Timberwolves,2.42
2,2024-04-05T14:00:39Z,1439dd82b2ac2434f097380dfa7c39aa,FanDuel,Minnesota Timberwolves,2.48,Phoenix Suns,1.57
3,2024-04-05T14:00:39Z,1439dd82b2ac2434f097380dfa7c39aa,FanDuel,Phoenix Suns,1.57,Minnesota Timberwolves,2.48
4,2024-04-05T14:00:39Z,1439dd82b2ac2434f097380dfa7c39aa,BetMGM,Minnesota Timberwolves,2.45,Phoenix Suns,1.57
...,...,...,...,...,...,...,...
67,2024-04-05T14:00:39Z,fdb70e18e33666006d5f99ae0f78be05,DraftKings,New York Knicks,1.98,Chicago Bulls,1.85
68,2024-04-05T14:00:39Z,fdb70e18e33666006d5f99ae0f78be05,FanDuel,Chicago Bulls,1.85,New York Knicks,2.00
69,2024-04-05T14:00:39Z,fdb70e18e33666006d5f99ae0f78be05,FanDuel,New York Knicks,2.00,Chicago Bulls,1.85
70,2024-04-05T14:00:39Z,fdb70e18e33666006d5f99ae0f78be05,BetMGM,Chicago Bulls,1.83,New York Knicks,2.00


In [21]:
full['Timestamp'] = pd.to_datetime(full['Timestamp'])

utc_timezone = pytz.utc
est_timezone = pytz.timezone('US/Eastern')

full['Timestamp'] = full['Timestamp'].dt.tz_convert(est_timezone)
full['Timestamp'] = full['Timestamp'].dt.strftime('%Y-%m-%d')

In [22]:
full['Team_x'] = full['Team_x'].map(mapping)
full['Team_y'] = full['Team_y'].map(mapping)

In [23]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-05,1439dd82b2ac2434f097380dfa7c39aa,DraftKings,MIN,2.42,PHO,1.59
1,2024-04-05,1439dd82b2ac2434f097380dfa7c39aa,DraftKings,PHO,1.59,MIN,2.42
2,2024-04-05,1439dd82b2ac2434f097380dfa7c39aa,FanDuel,MIN,2.48,PHO,1.57
3,2024-04-05,1439dd82b2ac2434f097380dfa7c39aa,FanDuel,PHO,1.57,MIN,2.48
4,2024-04-05,1439dd82b2ac2434f097380dfa7c39aa,BetMGM,MIN,2.45,PHO,1.57
...,...,...,...,...,...,...,...
67,2024-04-05,fdb70e18e33666006d5f99ae0f78be05,DraftKings,NYK,1.98,CHI,1.85
68,2024-04-05,fdb70e18e33666006d5f99ae0f78be05,FanDuel,CHI,1.85,NYK,2.00
69,2024-04-05,fdb70e18e33666006d5f99ae0f78be05,FanDuel,NYK,2.00,CHI,1.85
70,2024-04-05,fdb70e18e33666006d5f99ae0f78be05,BetMGM,CHI,1.83,NYK,2.00


In [24]:
fanduel = full[full['Sportsbook'] == 'FanDuel']
columns = fanduel.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Fanduel_odds_x','Team_y', 'Fanduel_odds_y']
fanduel.columns = columns
fanduel = fanduel.drop('Sportsbook', axis=1)

draftkings = full[full['Sportsbook'] == 'DraftKings']
columns = draftkings.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Draftkings_odds_x','Team_y', 'Draftkings_odds_y']
draftkings.columns = columns
draftkings = draftkings.drop('Sportsbook', axis=1)

betMGM = full[full['Sportsbook'] == 'BetMGM']
columns = betMGM.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'BetMGM_odds_x','Team_y', 'BetMGM_odds_y']
betMGM.columns = columns
betMGM = betMGM.drop('Sportsbook', axis=1)

In [25]:
merged_fd_dk = pd.merge(fanduel, draftkings, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

# Merge the result with betMGM DataFrame
final = pd.merge(merged_fd_dk, betMGM, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

In [26]:
columns = final.columns
columns = ['Timestamp', 'Id', 'Teams_x', 'Fanduel_odds_x','Teams_y', 'Fanduel_odds_y', 'Draftkings_odds_x', 'Draftkings_odds_y', 'BetMGM_odds_x', 'BetMGM_odds_y']
final.columns = columns
final = final.drop('Id', axis=1)

order = ['Timestamp', 'Teams_x', 'Fanduel_odds_x','Draftkings_odds_x', 'BetMGM_odds_x', 'Teams_y', 'Fanduel_odds_y','Draftkings_odds_y', 'BetMGM_odds_y']
final = final[order]

In [27]:
final

,Timestamp,Teams_x,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Teams_y,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y
0,2024-04-05,MIN,2.48,2.42,2.45,PHO,1.57,1.59,1.57
1,2024-04-05,PHO,1.57,1.59,1.57,MIN,2.48,2.42,2.45
2,2024-04-05,DAL,1.52,1.51,1.52,GSW,2.62,2.64,2.63
3,2024-04-05,GSW,2.62,2.64,2.63,DAL,1.52,1.51,1.52
4,2024-04-05,MIL,1.08,1.08,1.08,TOR,8.50,8.50,8.50
5,2024-04-05,TOR,8.50,8.50,8.50,MIL,1.08,1.08,1.08
6,2024-04-05,DET,2.00,1.98,2.00,MEM,1.85,1.85,1.83
7,2024-04-05,MEM,1.85,1.85,1.83,DET,2.00,1.98,2.00
8,2024-04-05,LAC,1.11,1.12,1.12,UTA,7.10,6.75,6.50
9,2024-04-05,UTA,7.10,6.75,6.50,LAC,1.11,1.12,1.12


In [28]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
whole = pd.read_csv(file_path, index_col=0)

In [29]:
complete = pd.concat([whole, final], axis=0)

In [30]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
complete.to_csv(file_path)